In [1]:
# !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech
# !pip install python-socketio[asyncio]

In [2]:
from tools.initialize_cerebras import init_cerebras
# from tools.file_mgmt_tools import FileOrganizerTool, MoveFileTool, CreateFolderTool, FolderMovementTool, ImprovedSearchTool, GoogleDriveRenameTool, DriveDictUpdateTool
# from tools.document_tools import GoogleDocWriteTool
# from tools.miscellaneous_mgmt import GmailSendPdfTool, GoogleSheetsUpdateTool, GoogleSheetsCreateTool

client,llm = init_cerebras()

In [3]:
llm

ChatCerebras(client=<openai.resources.chat.completions.Completions object at 0x000001D2D4D7B020>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D2D4D78590>, root_client=<openai.OpenAI object at 0x000001D2D4D78D10>, root_async_client=<openai.AsyncOpenAI object at 0x000001D2D4D7A360>, model_name='llama3.3-70b', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True, cerebras_api_key=SecretStr('**********'), cerebras_api_base='https://api.cerebras.ai/v1/', cerebras_proxy='')

In [4]:
from google.cloud import texttospeech
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    MessagesPlaceholder, 
    PromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain.agents import create_structured_chat_agent, AgentExecutor

from langchain_community.tools import HumanInputRun

import langchain_core
import typing

prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                    'You are a document management assistant proficient in using GSuite tools. '
                    'Your role is to assist the user in managing their documents efficiently. '
                    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
                    '\n\n IMPORTANT!!!!!!! - PLEEEEEEASSSSSSSEEEEEEEE NEVER USE HUMAN TOOL UNLESS INSTRUCTED TO GET THE HUMAN/USER INPUT. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
                    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

                    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
                    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
                    'String Length: Ensure no individual string exceeds 5000 bytes.'
                    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
                    
                    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
                    
                    
                    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
                    

                    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

                    'You have access to the following tools:\n\n{tools}\n\n'

                    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
                    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
                    
                    

                    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

                    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
                    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
                    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'Follow this format:\n\n'
                    'Question: input question to answer\n'
                    'Thought: consider previous and subsequent steps\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
                    'Thought: I know what to respond\n'
                    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
                    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
                    'Use tools if necessary and respond directly if appropriate. '
                    'Ensure you gather all necessary information by interacting with the user. '
                    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


human_prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                     
    'You are Marvin, an expert at questioning clients about their HVAC service needs to provide accurate quotes. When you speak for the first time, introduce yourself as Marvin. Ask the user specific information needed for the quote. Follow these guidelines:'

    '1. **Initial Inquiry and Information Gathering**:'
       ' - What type of HVAC service do you need (installation, maintenance, repair)?'
        '- What is the make and model of your current HVAC system?'
        '- Are there any specific issues or symptoms you are experiencing?'

    '2. **Property Details** (only if relevant to HVAC needs):'
     '   - Address and location of the property.'
      '  - Type of property (residential, commercial).'
       ' - Age and current condition of the property.'
       ' - Size of the home or area that needs heating/cooling.'
       ' - Number of rooms and their usage (e.g., bedrooms, office space).'

    '3. **System Details**:'
       ' - Age and efficiency rating of the existing HVAC system.'
        '- Any known problems with the current system.'
        '- Recent changes to the HVAC system.'

   ' 4. **Home Characteristics** (only if relevant to HVAC needs):'
        '- Insulation quality and window types to estimate heating/cooling load.'
        '- Any unique architectural features that may affect HVAC installation.'

    '5. **Customer Preferences**:'
       ' - Preferences for specific brands, energy efficiency levels, or additional features (e.g., smart thermostats, air purifiers).'
        '- Level of finishes desired (standard, premium, luxury).'

    '6. **Budget**:'
        '- Your budget range for the project.'
        '- Any flexibility within the budget.'

    '7. **Timeline**:'
        '- Desired start date and completion date.'
        '- Any constraints or deadlines (e.g., events planned at the property).'

   

    'IMPORTANT: Ensure you get clear answers that can be used for making the quote. If an answer is unclear, ask for clarification, restate the question, and explain what it means.'

    'IMPORTANT: Ask each question ONE BY ONE. When one question is answered move onto the next'

    'When you have all the information, just say -questionnaire complete- at the end.'


                    
    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
    '\n\n IMPORTANT!!!!!!! YOU CAN ONLY USE THE HUMAN TOOL. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
    'String Length: Ensure no individual string exceeds 5000 bytes.'
    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
    
    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
    
    
    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
    

    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

    'You have access to the following tools:\n\n{tools}\n\n'

    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
    
    

    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
    'Follow this format:\n\n'
    'Question: input question to answer\n'
    'Thought: consider previous and subsequent steps\n'
    'Action:\n```\n$JSON_BLOB\n```\n'
    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
    'Thought: I know what to respond\n'
    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
    'Use tools if necessary and respond directly if appropriate. '
    'Ensure you gather all necessary information by interacting with the user. '
    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


In [5]:
import pyttsx3
import os
import time
from IPython.display import Audio, display

class TTSManager:
    def __init__(self):
        self.engine = pyttsx3.init()
        voices = self.engine.getProperty('voices')
        self.engine.setProperty('voice', voices[0].id)
        self.engine.setProperty('rate', 175)
        self.engine.setProperty('volume', 1.0)
    
    def synthesize(self, text, output_path):
        try:
            # Ensure directory exists
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Create a temporary path in case of file lock issues
            temp_path = f'temp_speech_{int(time.time())}.wav'
            
            # Save to temp file first
            self.engine.save_to_file(text, temp_path)
            self.engine.runAndWait()
            
            # Move to final location
            if os.path.exists(temp_path):
                # If target file exists, remove it first
                if os.path.exists(output_path):
                    os.remove(output_path)
                os.rename(temp_path, output_path)
                print(f"Speech saved to: {output_path}")
                return output_path
            else:
                print("Failed to create speech file")
                return None
                
        except Exception as e:
            print(f"TTS Error: {str(e)}")
            # Cleanup temp file if it exists
            if os.path.exists(temp_path):
                os.remove(temp_path)
            return None

# Create global TTS manager
tts_manager = TTSManager()

# Test with specific path
tts_synthesis_path = os.path.join('paths', 'synthesis.wav')
test_result = tts_manager.synthesize("This is a test of saving to a specific path", tts_synthesis_path)
if test_result:
    print(f"Test successful - file saved to {test_result}")
    # Verify file exists and has content
    print(f"File size: {os.path.getsize(test_result)} bytes")

Speech saved to: paths\synthesis.wav
Test successful - file saved to paths\synthesis.wav
File size: 130092 bytes


In [6]:
import queue
from quart import Quart, request, jsonify, send_file, render_template
from flask import Flask
import whisper
import pyaudio
import webrtcvad
import collections
from gtts import gTTS  # Added gTTS import
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from quart_cors import cors
import logging
import os
from dotenv import load_dotenv
import socketio
from langchain.memory import ConversationBufferMemory

from Utils_cerebras import (
    initialize_web_search_agent,
    initialize_pdf_search_agent,
    run_quote_logics,
    vector_embedding
)

load_dotenv()

import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)  # Male voice
engine.setProperty('rate', 175)     # Speaking rate
engine.setProperty('volume', 1.0)   # Volume level
    
# Initialize configuration
audio_path = os.getenv('AUDIO_PATH', 'audio.wav')  # Default paths if not set
tts_synthesis_path = os.getenv('NEW_TTS_SYNTHESIS', 'speech.wav')

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Initialize Quart app and Socket.IO
app = Quart(__name__)
app = cors(app, allow_origin="*")
sio = socketio.AsyncServer(async_mode='asgi', cors_allowed_origins="*")
app_asgi = socketio.ASGIApp(sio, app)

# Initialize Flask app for sync operations
app_sync = Flask(__name__)
sio_sync = socketio.Server()
app_sync.wsgi_app = socketio.WSGIApp(sio_sync, app_sync.wsgi_app)

# Initialize memory
chat_history = ConversationBufferMemory()

# Load Whisper model
model = whisper.load_model("base")

# Audio configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
is_recording = False

# Initialize audio components
audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(3)
executor = ThreadPoolExecutor(max_workers=20)

# Initialize human response queue
human_response_queue = queue.Queue()

@app.route('/start_recording', methods=['POST'])
async def start_recording():
    global is_recording
    if is_recording:
        return jsonify({"status": "Recording is already in progress"}), 400
    is_recording = True
    asyncio.create_task(record_audio())
    return jsonify({"status": "Recording started"})

@app.route('/stop_recording', methods=['POST'])
async def stop_recording():
    global is_recording
    if not is_recording:
        return jsonify({"status": "No recording in progress"}), 400
    is_recording = False
    return jsonify({"status": "Recording stopped"})

async def record_audio(**kwargs):
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)
            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i * 320:(i + 1) * 320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        async with aiofiles.open(audio_path, 'wb') as wf:
            await wf.write(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    try:
        result = model.transcribe(audio_path)
        transcription = result['text']
        logger.debug(f'Audio transcription completed: {transcription}')
        return transcription
    except Exception as e:
        logger.error(f"An error occurred during transcription: {e}")
        return ""

# async def ai_response(transcription: str):
#         
#     logger.debug(f'Generating AI response for transcription: {transcription}')
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content": f"""
#                 You are Marvin, a research expert who specializes in anything using web search and answering the
#                 users' questions. You can also do HVAC related things as per guidelines below but besides that you adapt to what user says.
#                 Ask the user specific information needed for the quote. 
#                 Follow these guidelines:
#                 1. Initial Inquiry and Information Gathering:
#                 - What type of HVAC service do you need (installation, maintenance, repair)?
#                 - What is the make and model of your current HVAC system?
#                 - Are there any specific issues or symptoms you are experiencing?
#                     - ONLY WHEN USER TELLS YOU ISSUES / SYMPTOMS YOU MUST SAY 'I WILL INSTRUCT THE TECHNICIAN AGENT' - THESE EXACT WORDS IN YOUR IMMEDIATE RESPONSE.
#                 2. Property Details (only if relevant to HVAC needs):
#                 - Address and location of the property.
#                 - Type of property (residential, commercial).
#                 - Age and current condition of the property.
#                 - Size of the home or area that needs heating/cooling.
#                 - Number of rooms and their usage.
#                 3. System Details:
#                 - Age and efficiency rating of the existing HVAC system.
#                 - Any known problems with the current system.
#                 - Recent changes to the HVAC system.
#                 4. Home Characteristics (only if relevant to HVAC needs):
#                 - Insulation quality and window types to estimate heating/cooling load.
#                 - Any unique architectural features that may affect HVAC installation.
#                 5. Customer Preferences:
#                 - Preferences for specific brands, energy efficiency levels, or additional features.
#                 - Level of finishes desired (standard, premium, luxury).
#                 6. Budget:
#                 - Your budget range for the project.
#                 - Any flexibility within the budget.
#                 7. Timeline:
#                 - Desired start date and completion date.
#                 - Any constraints or deadlines.
#                 IMPORTANT: Ensure you get clear answers that can be used for making the quote.
#                 """
#             },
#             {
#                 "role": "user",
#                 "content": transcription + "\n\nHere is the chat history for context: [" + str(chat_history.buffer) + "]"
#             }
#         ],
#         model="llama3.3-70b",
#         temperature=0.5
#     )

#     response = chat_completion.choices[0].message.content
    
#     logger.debug(f'AI response generated: {response}')
    
#     await chat_history.asave_context({"input": transcription}, {"output": response})
#     logger.debug('INSIDE THE MEMORY: %s', chat_history.buffer)

#     # Changed socketio.emit to await sio.emit
#     await sio.emit('new_message', {'message': response, 'sender': 'bot'})

#     if 'i will instruct the technician agent' in response.lower():
#         res1 = await asyncio.create_task(initialize_pdf_search_agent(llm, "DO A EXPANSIVE WEB SEARCH AND FAMILIARIZE YOURSELF ALL ABOUT HVAC SPECIFICS SO THAT YOU CAN DIAGNOSE THE PROBLEMS MENTIONED HERE: " + transcription, vectors, chat_history=chat_history))
        
#         await chat_history.asave_context({"input": "DO A EXPANSIVE WEB SEARCH AND FAMILIARIZE YOURSELF ALL ABOUT HVAC SPECIFICS SO THAT YOU CAN DIAGNOSE THE PROBLEMS MENTIONED HERE: " + transcription}, {"output": res1})
        
#         # Changed socketio.emit to await sio.emit
#         await sio.emit('new_message', {'message': res1, 'sender': 'bot'}) 
    
#     if 'questionnaire' in response.lower() and 'complete' in response.lower():
#         task = asyncio.create_task(run_quote_logics(client, llm, chat_history=chat_history))
#         corrected_quote_result = await task
#         response = f"Can you please put this quote into a google doc [{corrected_quote_result}], move it to the folder called 'HVAC Quote Documents'. Then, put all the appointment related stuff and the link to the quote document into calendar (appointment calendar tool). PLEASE FOLLOW THE DIRECTIONS!!"
#         task2 = asyncio.create_task(handle_response_with_agents(response))
#         text = await task2
        
#     return response

async def ai_response(transcription: str):
    
    system_prompt = f"""
    You are Marvin, a highly intelligent assistant specializing in HVAC services. You are capable of researching and answering any question using web search.

    CRITICAL INSTRUCTIONS - READ CAREFULLY:

    - When the user mentions issues or symptoms related to HVAC systems, you MUST include the phrase 'I will instruct the technician agent' in your immediate response.
    - WHEN USER ASKS FOR COST OR WHEN COST MUST BE DETERMINED, YOU MUST SAY 'determining the HVAC quote'.
    - When you have gathered all the necessary information for a quote, you MUST say 'Questionnaire complete' at the end of your response.
    - IF USER DOES NOT SAY ANYTHING ABOUT HVAC, YOU MUST SAY 'forwarding to web search agent' BUT HANDLE ALL CONVERSATIONS NORMALLY. BE NATURAL. RESPOND TO THE GREETINGS!
    - DO NOT MENTION CUTOFF DATE OR INABILITY TO DO ANYTHING. YOU HAVE ACCESS TO REAL TIME WEB SEARCH!!!!! SEARCH IT UP ALL THE TIME! NEVER RELY ON YOUR OWN KNOWLEDGE.

    Remember to always be helpful, adaptive, and thorough in your responses.
    """

    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "system",
                "content": system_prompt  
            },
            {
                "role": "user",
                "content": transcription + "\n\nHere is the chat history for context: [" + str(chat_history.buffer) + "]"
            }
        ],
        model="llama3.3-70b",
        temperature=0.5
    )
    response = chat_completion.choices[0].message.content
    output_path = await asyncio.get_event_loop().run_in_executor(executor, tts_manager.synthesize, response,tts_synthesis_path)
    if output_path:
        await sio.emit('tts_complete', {
            'message': 'TTS synthesis complete',
            'file_path': output_path
        })
    else:
        logger.error("Failed to synthesize speech")
    logger.debug(f'AI response generated: {response}')

    
    logger.debug('INSIDE THE MEMORY: %s', chat_history.buffer)

    await sio.emit('new_message', {'message': response, 'sender': 'bot'})

    if 'forwarding' in response.lower() or 'web search' in response.lower():
        text = await handle_response_with_agents(response)
        await sio.emit('new_message', {'message': text, 'sender': 'bot'})
        output_path = await asyncio.get_event_loop().run_in_executor(executor, tts_manager.synthesize, response, tts_synthesis_path)
        if output_path:
            await sio.emit('tts_complete', {
                'message': 'TTS synthesis complete',
                'file_path': output_path
            })
        else:
            logger.error("Failed to synthesize speech")

    if 'I will instruct the technician agent' in response:
        # Trigger the technician agent
        res1 = await initialize_pdf_search_agent(
            llm, 
            f"Diagnose the problems mentioned here: {transcription}", 
            vector_embedding(), 
            chat_history=chat_history
        )
        await chat_history.asave_context({"input": transcription}, {"output": res1})
        output_path = await asyncio.get_event_loop().run_in_executor(executor, tts_manager.synthesize, response, tts_synthesis_path)
        if output_path:
            await sio.emit('new_message', {'message': res1, 'sender': 'bot'})
        else:
            logger.error("Failed to synthesize speech")
        

    if 'determining the hvac quote' in response.lower():
        # Run quote logic
        corrected_quote_result = await run_quote_logics(client, llm, chat_history=chat_history)
        r = f"Please process this quote data: [{corrected_quote_result}]"

        text = await handle_response_with_agents(r)
        output_path = await asyncio.get_event_loop().run_in_executor(executor, tts_manager.synthesize, response, tts_synthesis_path)
        if output_path:
            await sio.emit('new_message', {'message': text, 'sender': 'bot'})
        else:
            logger.error("Failed to synthesize speech")
        #await sio.emit('new_message', {'message': text, 'sender': 'bot'})

    return response


async def handle_response_with_agents(response):
    llm.temperature = 0.5
    logger.debug(f'Processing response with agents: {response}')

    #response += "Here is extra info you will need (BUT YOU PROMISE TO NEVER SAY THEM OUT LOUD, NOT EVEN THE NAME -- UNLESS USER ASKS YOU FOR THEM. THESE WILL BE USED IN TOOLS): \nCredentials:\n" + str(credentials)

 
    
    agent_executor = initialize_web_search_agent(llm)
    result = agent_executor.invoke({"input": response})
    sio.emit('finished_chain')
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))

    final_response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "please sanitize this input into SHORT SIMPLE sentences. IMPORTANT: NOTHING IN YOUR RESPONSE SHALL BE ENCLOSED IN ANY QUOTES!!!!!!! THE SANITIZED OUTPUT SHALL NOT BE PREFIXED BY ANYTHING. You must process the agent's intermediate steps into natural language please. An example: 'First, I did this. Then I did this etc etc etc' \n Here is the input that you need to process:\n " + mystr
            }
        ],
        model='llama3.3-70b',
    ).choices[0].message.content

    await chat_history.asave_context({"input": response}, {"output": final_response})
    
    return final_response

async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    try:
        loop = asyncio.get_event_loop()
        synthesize_and_save(text)
        logger.debug('Speech synthesis completed and file saved.')
        await sio.emit('tts_complete', {'message': 'TTS synthesis complete', 'file_path': tts_synthesis_path})
    except Exception as e:
        logger.error(f"An error occurred during speech synthesis: {e}")



def synthesize_and_save(text):
    try:
        global engine
        # Create a new file path with timestamp to avoid conflicts
        timestamp = int(time.time())
        output_path = f'speech_{timestamp}.wav'
        
        # Save to file
        engine.save_to_file(text, output_path)
        engine.runAndWait()
        
        # Move file to desired location
        import shutil
        shutil.move(output_path, tts_synthesis_path)
        
        logger.debug(f'Speech successfully synthesized and saved to {tts_synthesis_path}')
        return True
    except Exception as e:
        logger.error(f"TTS synthesis failed: {e}")
        return False

def synth_speech(text, output_file=None):
    logger.debug(f'Starting speech synthesis for text: {text}')
    try:
        global engine
        target_path = output_file if output_file else tts_synthesis_path
        
        # Save to file
        success = synthesize_and_save(text)
        
        if success:
            # Create async tasks for emitting events
            loop = asyncio.get_event_loop()
            loop.create_task(sio.emit('tts_complete', {
                'message': 'TTS synthesis complete', 
                'file_path': target_path
            }))
            loop.create_task(sio.emit('new_message', {
                'message': text, 
                'sender': 'bot'
            }))
            logger.debug('Speech synthesis completed and events emitted')
        else:
            logger.error("Failed to synthesize speech")
            
    except Exception as e:
        logger.error(f"An error occurred during speech synthesis: {e}")

@app.route('/')
async def index():
    return await render_template('index.html')

@app.route('/talk', methods=['POST'])
async def talk():
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400

    logger.debug('Starting audio transcription...')
    transcription = await asyncio.get_event_loop().run_in_executor(executor, transcribe_audio)

    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)

    return jsonify({'response': ai_resp})

@app.route('/text_input', methods=['POST'])
async def text_input():
    data = await request.get_json()
    text = data.get('text', '')

    if not text:
        return jsonify({"error": "No text provided"}), 400

    logger.debug('Generating AI response...')
    ai_resp = await ai_response(text)
    return jsonify({'response': ai_resp})

@app.route('/get_audio')
async def get_audio():
    return await send_file(tts_synthesis_path, mimetype="audio/mp3")

@sio.event
async def connect(sid, environ, auth):
    logger.debug(f'Client connected: {sid}')

@sio.event
async def disconnect(sid):
    logger.debug(f'Client disconnected: {sid}')

# Human input handling
human_response_queue = queue.Queue()

# Update the web_prompt_func to handle errors
def web_prompt_func(prompt):
    try:
        synth_speech(prompt, output_file=tts_synthesis_path)
        return prompt
    except Exception as e:
        logger.error(f"Error in web_prompt_func: {e}")
        return prompt

def web_input_func():
    sio_sync.emit('request_human_input')
    return human_response_queue.get()

@app.before_serving
async def startup():
    global engine
    engine = pyttsx3.init()
    # Configure engine properties
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.setProperty('rate', 175)
    engine.setProperty('volume', 1.0)

@app.after_serving
async def shutdown():
    global engine
    engine.stop()
    del engine

@sio.event 
async def provide_human_input(sid, data):
    human_input = data.get('text', '')
    human_response_queue.put(human_input)
    await sio.emit('human_input_received', {'status': 'received'}, room=sid)

if __name__ == '__main__':
    import uvicorn
    config = uvicorn.Config(app_asgi, host="127.0.0.1", port=5000, log_level="info")
    server = uvicorn.Server(config)

    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(server.serve())
    else:
        loop.run_until_complete(server.serve())


C:\Users\ncvn\AppData\Local\Temp\ipykernel_23616\1709927677.py:60: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_history = ConversationBufferMemory()


INFO:     Started server process [23616]
INFO:     Waiting for application startup.
2025-01-31 08:38:54,420 [MainThread] DEBUG: wrap_outparam(<POINTER(ISpeechObjectTokens) ptr=0x1d2d5035f60 at 1d2e2be12d0>)
2025-01-31 08:38:54,420 [MainThread] DEBUG: Release <POINTER(IUnknown) ptr=0x1d2d7f597f0 at 1d2e2be1350>
2025-01-31 08:38:54,423 [MainThread] DEBUG: Release <POINTER(ISpeechObjectTokens) ptr=0x1d2d5035f60 at 1d2e2be12d0>
2025-01-31 08:38:54,424 [MainThread] DEBUG: wrap_outparam(<POINTER(IDispatch) ptr=0x1d2d7f50950 at 1d2e2be18d0>)
2025-01-31 08:38:54,426 [MainThread] DEBUG: GetBestInterface(<POINTER(IDispatch) ptr=0x1d2d7f50950 at 1d2e2be18d0>)
2025-01-31 08:38:54,427 [MainThread] DEBUG: Does NOT implement IProvideClassInfo, trying IProvideClassInfo2
2025-01-31 08:38:54,428 [MainThread] DEBUG: Does NOT implement IProvideClassInfo/IProvideClassInfo2
2025-01-31 08:38:54,429 [MainThread] DEBUG: Release <POINTER(IUnknown) ptr=0x1d2d3ff71c0 at 1d2e2be2150>
2025-01-31 08:38:54,430 [MainT

Speech saved to: paths/synthesis.wav
RUNNING initialize_web_search_agent


> Entering new AgentExecutor chain...


2025-01-31 08:38:57,829 [MainThread] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 31 Jan 2025 13:38:57 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'x-request-id', b'90aa1ae26f770841-IAD'), (b'x-ratelimit-limit-requests-day', b'14400'), (b'x-ratelimit-limit-tokens-minute', b'60000'), (b'x-ratelimit-remaining-requests-day', b'14398'), (b'x-ratelimit-remaining-tokens-minute', b'59662'), (b'x-ratelimit-reset-requests-day', b'37262.85593175888'), (b'x-ratelimit-reset-tokens-minute', b'2.8559317588806152'), (b'referrer-policy', b'strict-origin-when-cross-origin'), (b'x-content-type-options', b'nosniff'), (b'strict-transport-security', b'max-age=3600; includeSubDomains'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=lNY4xY.dIIN.thI.paGLj_CYE49kx7AkL02FRNEC.Ao-1738330737-1.0.1.1-WF0jq2Hs7LAN7b4Wrf_my5qoQQ9nHx7_4s958z0vaWIPveauUXPfB12nRpUH8

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "Introduction to HVAC systems"
}
```

...

2025-01-31 08:43:20,153 [MainThread] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 31 Jan 2025 13:43:19 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'x-request-id', b'90aa2146cf7de643-SJC'), (b'x-ratelimit-limit-requests-day', b'14400'), (b'x-ratelimit-limit-tokens-minute', b'60000'), (b'x-ratelimit-remaining-requests-day', b'14384'), (b'x-ratelimit-remaining-tokens-minute', b'51214'), (b'x-ratelimit-reset-requests-day', b'37000.46577787399'), (b'x-ratelimit-reset-tokens-minute', b'40.46577787399292'), (b'referrer-policy', b'strict-origin-when-cross-origin'), (b'x-content-type-options', b'nosniff'), (b'strict-transport-security', b'max-age=3600; includeSubDomains'), (b'cf-cache-status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'90aa2146cf7de643-IAD'), (b'Content-Encoding', b'br'), (b'alt-svc', b'h3=":443"; ma=86400')])
2025-01-31 08:43:20,158 [MainThr

INFO:     127.0.0.1:60957 - "GET /get_audio?1738330993905 HTTP/1.1" 200 OK


2025-01-31 08:43:20,170 [ThreadPoolExecutor-0_0] DEBUG: SAPI.SPFileStream -> {947812B3-2AE1-4644-BA86-9E90DED7EC91}
2025-01-31 08:43:20,170 [ThreadPoolExecutor-0_0] DEBUG: CoCreateInstance({947812B3-2AE1-4644-BA86-9E90DED7EC91}, clsctx=None, interface=None)
2025-01-31 08:43:20,173 [ThreadPoolExecutor-0_0] DEBUG: GetBestInterface(<POINTER(IUnknown) ptr=0x1d2d7ff28d0 at 1d2e2cb2550>)
2025-01-31 08:43:20,175 [ThreadPoolExecutor-0_0] DEBUG: Does NOT implement IProvideClassInfo, trying IProvideClassInfo2
2025-01-31 08:43:20,180 [ThreadPoolExecutor-0_0] DEBUG: Does NOT implement IProvideClassInfo/IProvideClassInfo2
2025-01-31 08:43:20,182 [ThreadPoolExecutor-0_0] DEBUG: Release <POINTER(IUnknown) ptr=0x1d2d3ff7270 at 1d3599ad450>
2025-01-31 08:43:20,184 [ThreadPoolExecutor-0_0] DEBUG: Default interface is {AF67F125-AB39-4E93-B4A2-CC2E66E182A7}
2025-01-31 08:43:20,188 [ThreadPoolExecutor-0_0] DEBUG: Release <POINTER(IUnknown) ptr=0x1d2d7ff28d0 at 1d3599addd0>
2025-01-31 08:43:20,191 [ThreadPo

Speech saved to: paths/synthesis.wav
INFO:     127.0.0.1:60955 - "POST /text_input HTTP/1.1" 200 OK
INFO:     127.0.0.1:60955 - "GET /get_audio?1738331000579 HTTP/1.1" 200 OK


INFO:     connection closed
2025-01-31 08:45:13,175 [MainThread] DEBUG: Client disconnected: X-opHiW9B5en9j4DAAAB
